<a href="https://colab.research.google.com/gist/rxwei/2e4f5213167d1fe868e74ade3055c493/demo-4-minigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MiniGo

## Install MiniGo via Swift Package Manager

In [0]:
%install-swiftpm-flags -c release -Xlinker -ltensorflow
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("io19"))' MiniGo

## Load pre-trained model

In [0]:
import TensorFlow
import MiniGo
import Foundation

In [0]:
system("""
    mkdir -p MiniGo
    gsutil cp 'gs://minigo-pub/v15-19x19/models/000939-heron.*' MiniGo/
    """)

In [0]:
let boardSize = 19
let modelConfig = ModelConfiguration(boardSize: boardSize)
var model = GoModel(configuration: modelConfig)

let reader = PythonCheckpointReader(path: "MiniGo/000939-heron")
model.load(from: reader)

## Configure game

In [0]:
let simulationCountForOneMove = 40

let config = GameConfiguration(
    size: boardSize,
    komi: 7.5,
    isVerboseDebuggingEnabled: false
)

let predictor = MCTSModelBasedPredictor(boardSize: boardSize, model: model)

let mctsConfiguration = MCTSConfiguration(
    gameConfiguration: config,
    simulationCountForOneMove: simulationCountForOneMove
)

## Play!

In [0]:
try playOneGame(
    gameConfiguration: config,
    participants: [
        MCTSPolicy(participantName: "black", predictor: predictor,
                   configuration: mctsConfiguration),
        MCTSPolicy(participantName: "white", predictor: predictor,
                   configuration: mctsConfiguration),
    ])